In [1]:
from station_variants import STATION_VARIANTS
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from transformers import AutoTokenizer, AutoModel

# Обучаем предсказание станции

In [3]:
# Загружаем токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")

# Подготовка данных для обучения
texts = []  # Список текстов
labels = []  # Список меток

# Заполните texts и labels используя ваш датасет STATION_VARIANTS
for station, variants in STATION_VARIANTS.items():
    for variant in variants:
        texts.append(variant)
        labels.append(station)

# Кодирование меток
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [4]:
def get_embeddings(texts):
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=32)
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state[:, 0, :].numpy()  # Берем эмбеддинги первого токена [CLS]
    return embeddings

X = get_embeddings(texts)

In [5]:
# Обучение классификатора KNN
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X, labels_encoded)

# Функция для предсказания станции
def predict_station(query):
    query_embedding = get_embeddings([query])
    prediction = clf.predict(query_embedding)
    return label_encoder.inverse_transform(prediction)[0]

In [12]:
# Тестирование
print(predict_station("Баррикадной"))

Баррикадная


# Тестируем модель

In [7]:
import pandas as pd

# Пример данных
data = {
    "Вариант": [
        "Нижегородск", "Нижгородская", "Пыхтин", "Пихтно", "Аэропрт Внуково",
        "Аэрпорт Внукво", "Яхромкая", "Яхрмская", "Лианзов", "Лианзо",
        "Фзтек", "Физ тех", "Текстильщ", "Текстльщик", "Кнцевская", "Кутуз",
        "Сокл", "Щёлковской", "Окская", "Лефортов"
    ],
    "Правильное название": [
        "Нижегородская", "Нижегородская", "Пыхтино", "Пыхтино", "Аэропорт Внуково",
        "Аэропорт Внуково", "Яхромская", "Яхромская", "Лианозово", "Лианозово",
        "Физтех", "Физтех", "Текстильщики", "Текстильщики", "Кунцевская", "Кутузовская",
        "Сокол", "Щёлковская", "Окская", "Лефортово"
    ]
}

# Создание DataFrame
df = pd.DataFrame(data)

In [8]:
df['predicted'] = df['Вариант'].apply(predict_station)

In [9]:
df['Правильное название'] == df.predicted

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15    False
16     True
17     True
18     True
19     True
dtype: bool

# Сохраняем предсказывающую модель в файл

In [10]:
from joblib import dump

In [11]:
# Сохраняем классификатор KNN и LabelEncoder
dump(clf, 'stations_knn_classifier.joblib')
dump(label_encoder, 'stations_label_encoder.joblib')

# Сохраняем токенизатор
tokenizer.save_pretrained('stations_tokenizer')

# Сохраняем модель трансформера
model.save_pretrained('stations_rubert-tiny-model')